In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)
from scipy.stats import mode
from scipy import stats
import feather
import re
import warnings
import random

from IPython.display import clear_output

In [2]:
new_transactions = pd.read_csv('C:\\kdata\\new_merchant_transactions.csv', parse_dates=['purchase_date'])
historical_transactions = pd.read_csv('C:\\kdata\\historical_transactions.csv', parse_dates=['purchase_date'])

In [3]:
merchants = pd.read_csv('C:\\kdata\\merchants.csv')

In [4]:
raw_all_transactions = pd.concat([new_transactions,historical_transactions],axis=0)

In [5]:
raw_all_transactions.merchant_category_id.replace({-1:404},inplace=True)

In [6]:
raw_all_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [15]:
merchantcatidxcard = raw_all_transactions.groupby(['merchant_category_id']).agg({'card_id':'nunique','merchant_id':'nunique'})

In [20]:
merchantcatidxcard 

,card_id,merchant_id
merchant_category_id,,
2,32996,2159
9,7745,1270
11,23,4
14,6036,356
16,739,116
18,4,2
19,76334,2858
21,2421,336
27,23,6


In [26]:
for col in merchantcatidxcard:
    merchantcatidxcard[col+'_percent_to_mc_id'] = merchantcatidxcard[col]/merchantcatidxcard[col].sum()

In [35]:
merchantcatidxcard.columns = ['mci'+x for x in merchantcatidxcard]

In [16]:
merchantidxcard = raw_all_transactions.groupby(['merchant_id']).agg({'card_id':'nunique','merchant_category_id':'nunique'})

In [29]:
for col in merchantidxcard:
    merchantidxcard[col+'_percent_to_merchid'] = merchantidxcard[col]/merchantidxcard[col].sum()

In [38]:
merchantidxcard.columns = ['mi_'+x for x in merchantidxcard]

In [39]:
merchantidxcard.head()

,mi_card_id,mi_merchant_category_id,mi_card_id_percent_to_merchid,mi_merchant_category_id_percent_to_merchid
merchant_id,,,,
M_ID_000025127f,7,1,5.418997e-07,0.000002
M_ID_0000699140,50,1,3.870712e-06,0.000002
M_ID_00006a5552,1,1,7.741424e-08,0.000002
M_ID_000087311e,17,2,1.316042e-06,0.000004
M_ID_0000ab0b2d,26,2,2.012770e-06,0.000004


In [36]:
merchantcatidxcard

,mcicard_id,mcimerchant_id,mcicard_id_percent_to_mc_id,mcimerchant_id_percent_to_mc_id
merchant_category_id,,,,
2,32996,2159,4.904329e-03,0.004426
9,7745,1270,1.151171e-03,0.002603
11,23,4,3.418583e-06,0.000008
14,6036,356,8.971551e-04,0.000730
16,739,116,1.098406e-04,0.000238
18,4,2,5.945362e-07,0.000004
19,76334,2858,1.134583e-02,0.005858
21,2421,336,3.598430e-04,0.000689
27,23,6,3.418583e-06,0.000012


In [40]:
raw_all_transactions = pd.merge(raw_all_transactions,merchantidxcard,how='left',on='merchant_id')

In [41]:
raw_all_transactions = pd.merge(raw_all_transactions,merchantcatidxcard,how='left',on='merchant_category_id')

In [46]:
raw_all_transactions = raw_all_transactions[['card_id'] + list(raw_all_transactions.columns.values[-8:]) + ['state_id','subsector_id']]

In [51]:
raw_all_transactions.state_id.replace({-1:25},inplace=True)
raw_all_transactions.subsector_id.replace({-1:42},inplace=True)

In [61]:
def add_mode(x):
    return mode(x)[0]

In [63]:
groupped = raw_all_transactions.groupby(['card_id']).agg({x:['min','mean','max','std',add_mode] for x in raw_all_transactions if x!='card_id'})

In [64]:
groupped.head()

mi_card_id                                                \
                       min         mean       max           std add_mode   
card_id                                                                    
C_ID_00007093c1        1.0  3775.291391  239115.0  19671.554417     74.0   
C_ID_0001238066        4.0  5367.777027  239115.0  22266.461542    141.0   
C_ID_0001506ef0        2.0  8548.059701  239115.0  40796.485182   5221.0   
C_ID_0001793786        1.0  7170.582996  239115.0  37320.697354    758.0   
C_ID_000183fdda        2.0  9603.025806  239115.0  43233.979446   1140.0   

                mi_merchant_category_id                                      \
                                    min      mean   max        std add_mode   
card_id                                                                       
C_ID_00007093c1                     1.0  3.072848  14.0   3.585339      1.0   
C_ID_0001238066                     1.0  7.000000  72.0  16.438713      1.0   
C_ID_0001506ef0                     1.0  3.597015  26.0   5.648973      1.0   
C_ID_0001793786                     1.0  3.105263  76.0   7.639559      1.0   
C_ID_000183fdda                     1.0  1.929032  10.0   2.013299      1.0   

                mi_card_id_percent_to_merchid                                \
                                          min      mean       max       std   
card_id                                                                       
C_ID_00007093c1                  7.741424e-08  0.000292  0.018511  0.001523   
C_ID_0001238066                  3.096570e-07  0.000416  0.018511  0.001724   
C_ID_0001506ef0                  1.548285e-07  0.000662  0.018511  0.003158   
C_ID_0001793786                  7.741424e-08  0.000555  0.018511  0.002889   
C_ID_000183fdda                  1.548285e-07  0.000743  0.018511  0.003347   

                          mi_merchant_category_id_percent_to_merchid  \
                 add_mode                                        min   
card_id                                                                
C_ID_00007093c1  0.000006                                   0.000002   
C_ID_0001238066  0.000011                                   0.000002   
C_ID_0001506ef0  0.000404                                   0.000002   
C_ID_0001793786  0.000059                                   0.000002   
C_ID_000183fdda  0.000088                                   0.000002   

                                                        mcicard_id  \
                     mean       max       std  add_mode        min   
card_id                                                              
C_ID_00007093c1  0.000006  0.000029  0.000007  0.000002       3975   
C_ID_0001238066  0.000014  0.000148  0.000034  0.000002       4073   
C_ID_0001506ef0  0.000007  0.000053  0.000012  0.000002      11352   
C_ID_0001793786  0.000006  0.000156  0.000016  0.000002        125   
C_ID_000183fdda  0.000004  0.000020  0.000004  0.000002       3028   

                                                               mcimerchant_id  \
                          mean     max            std add_mode            min   
card_id                                                                         
C_ID_00007093c1  171033.410596  293019  100041.131494   234923             75   
C_ID_0001238066  197541.906040  293019   82197.659645   234923             44   
C_ID_0001506ef0  186176.264706  293019  101559.095723   293019             75   
C_ID_0001793786  152635.336032  293019   97173.555518   220721              8   
C_ID_000183fdda  158774.825806  293019   87640.244036   249632             75   

                                                             \
                         mean    max           std add_mode   
card_id                                                       
C_ID_00007093c1  15437.357616  35239  12040.797066    17843   
C_ID_0001238066  17699.926174  35239  11142.299620    17843   
C_ID_0001506ef0  18881.941176  35239  1

In [66]:
groupped.columns = ['_'.join(x) for x in groupped]

In [70]:
feather.write_dataframe(groupped,'merchants_and_city_id\\groupped')